In [1]:
import os
os.chdir('/home/dennis/SourceCode/NCTU-Machine-Learning/homework#1-2018Spring/libsvm-master/python')

In [2]:
import svm
import svmutil
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (10.0, 10.0)
%matplotlib inline
os.chdir('/home/dennis/SourceCode/NCTU-Machine-Learning/homework#1-2018Spring')

Data Preprocessing

In [3]:
X_train = pd.read_csv('./data/X_train.csv', header=None)
T_train = pd.read_csv('./data/T_train.csv', header=None)
X_test = pd.read_csv('./data/X_test.csv', header=None)
T_test = pd.read_csv('./data/T_test.csv', header=None)
print(X_train.shape)
print(T_train.shape)
print(X_test.shape)
print(T_test.shape)

(5000, 784)
(5000, 1)
(2500, 784)
(2500, 1)


In [4]:
X_train_dict = X_train.to_dict('records')
X_test_dict = X_test.to_dict('records')

In [5]:
T_train_list = T_train[0].tolist()
T_test_list = T_test[0].tolist()

Q1: Use different kernel functions (linear, polynomial, and RBF kernels) and have comparison between their performance. 

In [6]:
svm_prob = svm.svm_problem(T_train_list, X_train_dict)

Linear kernel

In [7]:
svm_param = svm.svm_parameter('-t 0 -b 1')
model_linear = svmutil.svm_train(svm_prob, svm_param)
p_label, p_acc, p_val = svmutil.svm_predict(T_test_list, X_test_dict, model_linear)

Model supports probability estimates, but disabled in predicton.
Accuracy = 95.08% (2377/2500) (classification)


Polynomial kernel

In [8]:
svm_param = svm.svm_parameter('-t 1 -b 1')
model_poly = svmutil.svm_train(svm_prob, svm_param)
p_label, p_acc, p_val = svmutil.svm_predict(T_test_list, X_test_dict, model_poly)

Model supports probability estimates, but disabled in predicton.
Accuracy = 34.72% (868/2500) (classification)


RBF kernel

In [9]:
svm_param = svm.svm_parameter('-t 2 -b 1')
model_rbf = svmutil.svm_train(svm_prob, svm_param)
p_label, p_acc, p_val = svmutil.svm_predict(T_test_list, X_test_dict, model_rbf)

Model supports probability estimates, but disabled in predicton.
Accuracy = 95.32% (2382/2500) (classification)


Q2: Please use C-SVC (you can choose by setting parameters in the function input, C-SVC is soft-margin SVM). Since there are some parameters you need to tune for, please do the grid search for finding parameters of best performing model. For instance, in C-SVC you have a parameter C, and if you use RBF kernel you have another parameter γ, you can search for a set of (C,γ) which gives you best performance in cross-validation. (lots of sources on internet, just google for it)

Parameter list(gamma and C)

In [10]:
C = [10 ** d for d in range(-4, 5)]
gamma = [10 ** d for d in range(-4, 5)]

4-fold cross-validation

In [11]:
def CrossValidation(C, gamma, fold, full_data, full_target):
#     fold_size = len(full_data) // fold
    svm_param = svm.svm_parameter('-t 2 -c {} -g {} -v {}'.format(C, gamma, fold))
    svm_prob = svm.svm_problem(full_target, full_data)
    acc = svmutil.svm_train(svm_prob, svm_param)
    return acc
#     for f in range(fold):
#         # Prepare training data and validation data
#         X_val = full_data[fold * fold_size: (fold+1) * fold_size]
#         T_val = full_target[fold * fold_size: (fold+1) * fold_size]
#         X_train = full_data[0 * fold_size : fold * fold_size] + full_data[(fold+1) * fold_size:]
#         T_train = full_target[0 * fold_size : fold * fold_size] + full_target[(fold+1) * fold_size:]
        
#         # Setup svm problem and model
#         svm_prob = svm.svm_problem(T_train, X_train)
#         model_rbf = svmutil.svm_train(svm_prob, svm_param)
        
#         _, p_acc, _ = 

In [12]:
best_pair = (0, 0)
best_acc = 0
for cost in C:
    for g in gamma:
        acc = CrossValidation(cost, g, 4, X_train_dict, T_train_list)
        if acc > best_acc:
            best_acc = acc
            best_pair = (cost, g)
print('Best parameter pair is ({}, {})'.format(best_pair[0], best_pair[1]))
print('Best Acc: {}'.format(best_acc))

Cross Validation Accuracy = 79.58%
Cross Validation Accuracy = 81.1%
Cross Validation Accuracy = 89.88%
Cross Validation Accuracy = 48.14%
Cross Validation Accuracy = 20.68%
Cross Validation Accuracy = 78.8%
Cross Validation Accuracy = 36.02%
Cross Validation Accuracy = 20%
Cross Validation Accuracy = 20%
Cross Validation Accuracy = 79.58%
Cross Validation Accuracy = 80.94%
Cross Validation Accuracy = 89.78%
Cross Validation Accuracy = 49.72%
Cross Validation Accuracy = 20.36%
Cross Validation Accuracy = 78.92%
Cross Validation Accuracy = 36.14%
Cross Validation Accuracy = 20%
Cross Validation Accuracy = 20%
Cross Validation Accuracy = 79.48%
Cross Validation Accuracy = 81.06%
Cross Validation Accuracy = 92.26%
Cross Validation Accuracy = 49.08%
Cross Validation Accuracy = 20.78%
Cross Validation Accuracy = 78.86%
Cross Validation Accuracy = 36.26%
Cross Validation Accuracy = 20%
Cross Validation Accuracy = 20%
Cross Validation Accuracy = 79.72%
Cross Validation Accuracy = 92.44%
Cross

Use best parameters to train the model

In [13]:
svm_param = svm.svm_parameter('-t 2 -c {} -g {}'.format(best_pair[0], best_pair[1]))
svm_prob = svm.svm_problem(T_train_list, X_train_dict)
model_rbf_best = svmutil.svm_train(svm_prob, svm_param)
p_label, p_acc, p_val = svmutil.svm_predict(T_test_list, X_test_dict, model_rbf_best)

Accuracy = 98.16% (2454/2500) (classification)


Q3: Use linear kernel+RBF kernel together (therefore a new kernel function) and compare its performance with respect to others. You would need to find out how to use a user-defined kernel in libsvm. 

Prepare user-defined kernel

In [14]:
from sklearn.metrics.pairwise import rbf_kernel, linear_kernel

In [15]:
X_train_mat = X_train.as_matrix()
X_test_mat = X_test.as_matrix()
print(X_train_mat.shape)
print(X_test_mat.shape)

(5000, 784)
(2500, 784)


In [16]:
custom_kernel = rbf_kernel(X_train_mat, gamma=best_pair[1]) + linear_kernel(X_train_mat)
print(custom_kernel.shape)

(5000, 5000)


In [17]:
custom_kernel_list = custom_kernel.tolist()
for i in range(len(custom_kernel_list)):
    custom_kernel_list[i].insert(0, i+1)

Train the model

In [18]:
# use parameters learned before
svm_param = svm.svm_parameter('-t 4 -c {} -g {} -b 1'.format(best_pair[0], best_pair[1]))
svm_prob = svm.svm_problem(T_train_list, custom_kernel_list, isKernel=True)
model = svmutil.svm_train(svm_prob, svm_param)

Create kernel between training data and testing data

In [19]:
test_kernel = rbf_kernel(X_test_mat, X_train_mat, gamma=best_pair[1]) + linear_kernel(X_test_mat, X_train_mat)
print(test_kernel.shape)

(2500, 5000)


In [20]:
test_kernel_list = test_kernel.tolist()
for i in range(len(test_kernel_list)):
    test_kernel_list[i].insert(0, i+1)

Test on the model

In [21]:
p_label, p_acc, p_val = svmutil.svm_predict(T_test_list, test_kernel_list, model)

Model supports probability estimates, but disabled in predicton.
Accuracy = 95.32% (2382/2500) (classification)
